# 使用 OpenBnB MCP 服务器集成的语义内核

本笔记展示了如何使用语义内核与实际的 OpenBnB MCP 服务器集成，通过 MCPStdioPlugin 搜索真实的 Airbnb 住宿。对于 LLM 访问，使用了 Azure AI Foundry。要设置您的环境变量，可以参考 [设置课程](/00-course-setup/README.md)。


## 导入所需的包


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## 创建 MCP 插件连接

我们将使用 MCPStdioPlugin 连接到 [OpenBnB MCP 服务器](https://github.com/openbnb-org/mcp-server-airbnb)。该服务器通过 @openbnb/mcp-server-airbnb 包提供 Airbnb 搜索功能。


## 创建客户端

在本示例中，我们将使用 Azure AI Foundry 来访问 LLM。请确保您的环境变量已正确设置。


## 环境配置

配置 Azure OpenAI 设置。确保已设置以下环境变量：
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## 了解 OpenBnB MCP 集成

此笔记本连接到**真实的 OpenBnB MCP 服务器**，提供实际的 Airbnb 搜索功能。

### 工作原理：

1. **MCPStdioPlugin**：通过标准输入/输出与 MCP 服务器通信  
2. **真实 NPM 包**：通过 npx 下载并运行 `@openbnb/mcp-server-airbnb`  
3. **实时数据**：从 Airbnb 的 API 返回真实的房源数据  
4. **功能发现**：代理会自动发现 MCP 服务器提供的可用功能  

### 可用功能：

OpenBnB MCP 服务器通常提供以下功能：  
- **search_listings** - 根据位置和条件搜索 Airbnb 房源  
- **get_listing_details** - 获取特定房源的详细信息  
- **check_availability** - 检查特定日期的可用性  
- **get_reviews** - 获取房源的评论  
- **get_host_info** - 获取房东的相关信息  

### 前置条件：

- 系统中已安装 **Node.js**  
- **互联网连接**，用于下载 MCP 服务器包  
- **NPX** 可用（Node.js 自带）  

### 测试连接：

您可以通过运行以下命令手动测试 MCP 服务器：  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

这将下载并启动 OpenBnB MCP 服务器，随后 Semantic Kernel 将连接到该服务器以获取真实的 Airbnb 数据。  


## 运行与 OpenBnB MCP 服务器连接的代理

现在我们将运行连接到 OpenBnB MCP 服务器的 AI 代理，以搜索斯德哥尔摩适合2名成人和1名儿童的真实Airbnb住宿。您可以随意修改 `user_inputs` 列表来更改搜索条件。


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

总结  
恭喜你！你已经成功构建了一个集成真实世界住宿搜索的AI代理，并使用了模型上下文协议（MCP）：  

使用的技术：  
- Semantic Kernel - 用于使用 Azure OpenAI 构建智能代理  
- Azure AI Foundry - 提供大语言模型功能和聊天补全  
- MCP（模型上下文协议） - 用于标准化工具集成  
- OpenBnB MCP Server - 提供真实的Airbnb搜索功能  
- Node.js/NPX - 用于运行外部MCP服务器  

你学到了什么：  
- MCP集成：将Semantic Kernel代理连接到外部MCP服务器  
- 实时数据访问：通过实时API搜索真实的Airbnb房源  
- 协议通信：使用标准输入输出在代理和MCP服务器之间进行通信  
- 功能发现：自动发现MCP服务器提供的可用功能  
- 流式响应：实时捕获和记录函数调用  
- HTML渲染：以样式化表格和交互式显示格式化代理响应  

下一步：  
- 集成更多MCP服务器（天气、航班、餐厅）  
- 构建结合MCP和A2A协议的多代理系统  
- 为自己的数据源创建自定义MCP服务器  
- 实现跨会话的持久对话记忆  
- 将代理部署到Azure Functions并进行MCP服务器编排  
- 添加用户认证和预订功能  

MCP架构的主要优势：  
- 标准化：连接AI代理到外部工具的通用协议  
- 实时数据：从各种服务获取实时、最新的信息  
- 可扩展性：轻松集成新的数据源和工具  
- 互操作性：适用于不同的AI框架和代理平台  
- 关注点分离：明确区分AI逻辑和外部数据访问  



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保准确性，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文档作为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而引起的任何误解或误读，我们概不负责。
